AARRR

A: acquisition

A: activation

R: retention

R: revenue

R: referral

## Acquisition

In [ ]:
# user sign-up
query = '''--sql
select
    date_trunc('month', signup_date) as month,
    count(user_id) as signups
from 
    users
group by 1
order by 1
;
'''


# funnel analysis
query = '''--sql

with funnel as (
    select
        user_id,
        min(case when event = 'visit' then event_date end) as visit_date,
        min(case when event = 'signup' then event_date end) as signup_date,
        min(case when event = 'purchase' then event_date end) as purchase_date
    from 
        events
    group by 1
)

select
    count(distinct user_id) as total_users,
    count(distinct case when visit_date is not null then user_id end) as visits,
    count(distinct case when signup_date is not null then user_id end) as signups,
    count(distinct case when purchase_date is not null then user_id end) as purchases

    visits * 100.0 / total_users as visit_rate,
    signups * 100.0 / total_users as signup_date,
    purchases * 100.0 / total_users as purchase_rate
from 
    funnel

;
'''



# retention by cohort
query = '''--sql

with 

cohort as (
    select 
        user_id,
        min(date_trunc('month', signup_date)) as cohort_month
    from 
        users
    group by 1
),

user_activity as (
    select
        distinct 
        user_id,
        date_trunc('month', activity_date) as activity_month
    from 
        user_activity
)

select
    cohort_month,
    activity_month,
    count(distinct ua.user_id) as active_users
from
    cohort c inner join user_activity ua on c.user_id = ua.user_id
group by 1, 2
order by 1, 2
;
'''






# DAU
query = '''--sql
select
    date_trunc('month', signup_date) as month,
    count(user_id) as signups
from 
    users
group by 1
order by 1
;
'''

# user sign-up
query = '''--sql
select
    date_trunc('month', signup_date) as month,
    count(user_id) as signups
from 
    users
group by 1
order by 1
;
'''




## Retention

In [ ]:
# DAU
query = '''--sql
select
    activity_date,
    count(distinct user_id) as dau
from 
    user_activity
group by 1
order by 1 asc
;
'''

# MAU
query = '''--sql
select
    date_trunc('month', activity_date) as month,
    count(distinct user_id) as mau
from 
    user_activity
group by 1
order by 1 asc
;
'''

# Retention rate
query = '''--sql
with

user_signups as (
    select
        user_id,
        min(signup_date) as signup_date
    from 
        users
    group by 1
),

user_activity as (
    select
        user_id,
        date_trunc('day', activity_date) as activity_date
    from 
        user_activity
)

select
    date_trunc('week', us.signup_date) as signup_week,
    date_trunc('week', ua.activity_date) as activity_week,
    count(distinct ua.user_id) as retained_users
from
    user_signups us inner join user_activity ua on us.user_id = ua.user_id 
where
    ua.activity_date >= us.signup_date
group by 1, 2
order by 1, 2
;
'''

## Revenue

In [ ]:
# total sales
query = '''--sql
select
    date_trunc('month', order_date) as month,
    count(order_id) as total_orders,
    sum(order_amount) as total_revenue
from
    orders
group by 1
order by 1
;
'''


# average order value 
query = '''--sql
select
    date_trunc('month', order_date) as month,
    avg(order_amount) as avg_order_value
from
    orders
group by 1
order by 1
;
'''

## Customer Lifetime Value (CLTV)

In [ ]:
# customer lifetime value
query = '''--sql

with customer_orders as (
    select
        customer_id,
        sum(order_amount) as total_spent,
        count(order_id) as total_orders,
        count(distinct date_trunc('month', order_date)) as total_months_with_orders
    from
        orders
    group by 1
)
select 
    customer_id,
    total_spent,
    total_orders,
    total_spent / total_orders as avg_order_value,
    total_spent / total_months_with_orders as monthly_cltv
from 
    customer_orders
order by 4 desc;

'''





